<a href="https://colab.research.google.com/github/Futaba-Kosuke/STL10/blob/develop/training/main_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
 !ls

drive  images  sample_data


In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image

# バージョン確認 (Google Colab default)
print(torch.__version__)  # 1.4.0
print(torchvision.__version__)  # 0.5.0
print(np.__version__)  # 1.18.2
print(matplotlib.__version__)  # 3.2.1
print(Image.__version__)  # 7.0.0

1.4.0
0.5.0
1.18.2
3.2.1
7.0.0


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
batch_size = 25
num_workers = 3

In [8]:
# データの読み込み
transform = {
  'train': transforms.Compose(
    [transforms.RandomHorizontalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
  ),
  'test': transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
  )
}
data_set = {
  x: torchvision.datasets.STL10(root='./images', split=x, download=True, transform=transform[x])
  for x in ('train', 'test')
}
data_size = {
  x: len(data_set[x]) for x in ('train', 'test')
}
print(data_size)

data_loaders = {
  x[0]: torch.utils.data.DataLoader(data_set[x[0]], batch_size=x[1], shuffle=x[2], num_workers=num_workers)
  for x in (('train', batch_size, True), ('test', 100, False))
}

Files already downloaded and verified
Files already downloaded and verified
{'train': 5000, 'test': 8000}


In [0]:
classes = ('airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')

In [10]:
"""
各モデルで10エポック学習して検証
resnet: 90 %
alexnet: 79 %
vgg16: 93 %
densenet: 94 %
googlenet: 90 %
shufflenet: 62 %
mobilenet: 89 %
resnext: 93 %
wide_resnet: 94 %
mnasnet: 85 %

vgg16, densenet, wide_resnetを一旦採用、各々学習させてモデルを出力させた。
"""

# モデル構築
from torchvision import models
from torch import nn

# net = models.vgg16(pretrained=True)
# last_in_features = net.classifier[6].in_features
# net.classifier[6] = nn.Linear(last_in_features, 10)

# net = models.densenet161(pretrained=True)
# last_in_features = net.classifier.in_features
# net.classifier = nn.Linear(last_in_features, 10)

net = models.wide_resnet50_2(pretrained=True)
last_in_features = net.fc.in_features
net.fc = nn.Linear(last_in_features, 10)

net = net.to(device)
net

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [0]:
# 損失・最適関数の定義
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # 10エポックごとに学習率が1/10に更新

In [0]:
# 学習用関数
def train_net(net, criterion, optimizer, scheduler, num_epochs):
  for epoch in range(num_epochs):
    print('Epoch %d/%d' % (epoch, num_epochs - 1))
    print('-' * 10)

    running_loss_sum = 0
    train_loss_sum = 0
    test_loss_sum = 0

    # モデルの更新
    net.train()
    for i, (inputs, labels) in enumerate(data_loaders['train']):
      inputs = inputs.to(device)
      labels = labels.to(device)
      # 勾配の初期化
      optimizer.zero_grad()
      # 予測
      outputs = net(inputs)
      # 損失の導出
      loss = criterion(outputs, labels)
      # 逆伝播
      loss.backward()
      # 勾配の更新
      optimizer.step()

      running_loss_sum += loss.item()
      train_loss_sum += loss.item()
      if i % 50 == 49:
        print('[%d] running_loss: %.6f' % (i + 1, running_loss_sum / 50))
        running_loss_sum = 0

    train_cnt = i

    # モデルの評価
    net.eval()
    cnt_correct = 0
    for i, (inputs, labels) in enumerate(data_loaders['test']):
      inputs = inputs.to(device)
      labels = labels.to(device)

      # 予測
      outputs = net(inputs)
      # 損失の導出
      loss = criterion(outputs, labels)
      # lossの加算
      test_loss_sum += loss.item()

      _, predicted = torch.max(outputs, 1)
      is_correct = (predicted == labels).squeeze()
      
      for j in range(len(is_correct)):
        cnt_correct += is_correct[j].item()  # 正解なら1, 不正解なら0

    test_cnt = i

    print('')
    print('train_loss_ave:\t%.6f' % (train_loss_sum / train_cnt))
    print('test_loss_ave:\t%.6f' % (test_loss_sum / test_cnt))
    print('test_accuracy: \t%.6f %%' % (cnt_correct / (test_cnt)))

    net.train()
    scheduler.step()

    if cnt_correct / (test_cnt) > 96:
      break

In [13]:
num_epochs = 100
train_net(net, criterion, optimizer, scheduler, num_epochs)

Epoch 0/99
----------
[50] running_loss: 1.798012
[100] running_loss: 0.839991
[150] running_loss: 0.603944
[200] running_loss: 0.537368

train_loss_ave:	0.949577
test_loss_ave:	0.278594
test_accuracy: 	92.417722 %
Epoch 1/99
----------
[50] running_loss: 0.447262
[100] running_loss: 0.375123
[150] running_loss: 0.299524
[200] running_loss: 0.334651

train_loss_ave:	0.365970
test_loss_ave:	0.229476
test_accuracy: 	93.569620 %
Epoch 2/99
----------
[50] running_loss: 0.240076
[100] running_loss: 0.288222
[150] running_loss: 0.281907
[200] running_loss: 0.253488

train_loss_ave:	0.267259
test_loss_ave:	0.207230
test_accuracy: 	94.506329 %
Epoch 3/99
----------
[50] running_loss: 0.235370
[100] running_loss: 0.182753
[150] running_loss: 0.186859
[200] running_loss: 0.179202

train_loss_ave:	0.197031
test_loss_ave:	0.182940
test_accuracy: 	95.379747 %
Epoch 4/99
----------
[50] running_loss: 0.184082
[100] running_loss: 0.178825
[150] running_loss: 0.166645
[200] running_loss: 0.161811

tr

In [14]:
net.eval()

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for (inputs, labels) in data_loaders['test']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = net(inputs)
    
    _, predicted = torch.max(outputs, 1)
    is_correct = (predicted == labels).squeeze()

    for i in range(len(is_correct)):
      label = labels[i]
      class_correct[label] += is_correct[i].item()  # 正解なら1, 不正解なら0
      class_total[label] += 1

for i in range(10):
  print('Accuracy of %5s : %2.01f %%' % (
    classes[i], 100 * class_correct[i] / class_total[i]))
  
print('Accuracy Ave: %2.01f %%' % (100 * sum(class_correct) / sum(class_total)))

Accuracy of airplane : 96.5 %
Accuracy of  bird : 95.5 %
Accuracy of   car : 97.8 %
Accuracy of   cat : 89.1 %
Accuracy of  deer : 93.9 %
Accuracy of   dog : 93.9 %
Accuracy of horse : 94.1 %
Accuracy of monkey : 96.4 %
Accuracy of  ship : 98.0 %
Accuracy of truck : 93.5 %
Accuracy Ave: 94.9 %


In [15]:
net.eval()

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for (inputs, labels) in data_loaders['train']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = net(inputs)
    
    _, predicted = torch.max(outputs, 1)
    is_correct = (predicted == labels).squeeze()

    for i in range(len(is_correct)):
      label = labels[i]
      class_correct[label] += is_correct[i].item()  # 正解なら1, 不正解なら0
      class_total[label] += 1

for i in range(10):
  print('Accuracy of %5s : %2.01f %%' % (
    classes[i], 100 * class_correct[i] / class_total[i]))
  
print('Accuracy Ave: %2.01f %%' % (100 * sum(class_correct) / sum(class_total)))

Accuracy of airplane : 99.6 %
Accuracy of  bird : 99.4 %
Accuracy of   car : 99.8 %
Accuracy of   cat : 98.0 %
Accuracy of  deer : 99.4 %
Accuracy of   dog : 99.6 %
Accuracy of horse : 99.8 %
Accuracy of monkey : 99.2 %
Accuracy of  ship : 99.6 %
Accuracy of truck : 99.6 %
Accuracy Ave: 99.4 %


In [0]:
# 保存
PATH = './drive/My Drive/wide_resnet'
torch.save(net.state_dict(), PATH+'_gpu.pth')
torch.save(net.to('cpu').state_dict(), PATH+'_cpu.pth')